# Notebook to create invididual pages for web pages


In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

from coronavirus import *
pd.set_option('display.float_format', '{:.2f}'.format)  #  Disable pandas scientific notation
# force download of new data
# clear_cache()



In [ ]:
# fetch world data
d, c = fetch_deaths(), fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

In [ ]:
fig, ax =  plt.subplots(figsize=(12, 3))
cases, deaths = get_country_data("United Kingdom")
plot_daily_change(ax, cases, 'C1')
filename = 'fig-homepage-example-plot'
outdir = 'pelican/content/pages'
outpath = os.path.join(outdir, filename)
fig.savefig(outpath + '.svg')
# fig.savefig(outpath + '.png', dpi=250)